# **18.1 Understanding_Encapsulation**

Encapsulation means bundling data and methods together while controlling access to internal details. Instead of letting anyone set `pokemon.hp = -999`, encapsulation validates changes through controlled interfaces. In this lesson you'll learn the principles of encapsulation, why it matters, information hiding, and how to build Pokemon classes that protect their own data integrity.

---

## **The Problem: No Encapsulation**

Without encapsulation, internal data is exposed and can be corrupted.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp):
        self.name = name
        self.level = level
        self.hp = hp
        self.max_hp = hp

pikachu = Pokemon("Pikachu", 25, 35)

# Nothing prevents invalid states!
pikachu.hp = -999        # Invalid HP
pikachu.level = 0        # Invalid level
pikachu.max_hp = -50     # Nonsensical max HP
pikachu.name = ""        # Empty name

print(f"{pikachu.name} Lv.{pikachu.level} HP:{pikachu.hp}/{pikachu.max_hp}")
print("\n⚠️ Data integrity is broken!")

---

## **Encapsulation with Methods**

Control access through methods that validate input.

In [ ]:
class Pokemon:
    def __init__(self, name, level, hp):
        self.name = name
        self.level = level
        self.hp = hp
        self.max_hp = hp
    
    def take_damage(self, damage):
        """Controlled method to reduce HP."""
        if damage < 0:
            print("Damage cannot be negative")
            return
        
        self.hp = max(0, self.hp - damage)
        if self.hp == 0:
            print(f"{self.name} fainted!")
    
    def heal(self, amount):
        """Controlled method to restore HP."""
        if amount < 0:
            print("Healing amount cannot be negative")
            return
        
        self.hp = min(self.max_hp, self.hp + amount)
        print(f"{self.name} restored {amount} HP")

pikachu = Pokemon("Pikachu", 25, 35)

# Use controlled methods
pikachu.take_damage(10)
print(f"HP: {pikachu.hp}")

pikachu.heal(5)
print(f"HP: {pikachu.hp}")

# Validation prevents invalid input
pikachu.take_damage(-50)  # Rejected

# But direct access still possible!
pikachu.hp = -999  # Still not prevented
print(f"\n⚠️ Direct access bypasses validation: HP = {pikachu.hp}")

---

## **Information Hiding Principle**

Hide implementation details, expose only what's necessary.

In [ ]:
class Pokemon:
    """Pokemon with hidden internal state."""
    
    def __init__(self, name, level, hp):
        # Public interface
        self.name = name
        self.level = level
        
        # Internal state (should be private)
        self._hp = hp         # Convention: underscore = private
        self._max_hp = hp
        self._fainted = False
    
    # Public interface methods
    def get_hp(self):
        """Safe read access to HP."""
        return self._hp
    
    def is_fainted(self):
        """Check if Pokemon has fainted."""
        return self._fainted
    
    def take_damage(self, damage):
        """Controlled damage with validation."""
        if self._fainted:
            print(f"{self.name} is already fainted!")
            return
        
        self._hp = max(0, self._hp - damage)
        
        if self._hp == 0:
            self._fainted = True
            print(f"{self.name} fainted!")
    
    def __str__(self):
        status = "Fainted" if self._fainted else f"HP:{self._hp}/{self._max_hp}"
        return f"{self.name} Lv.{self.level} ({status})"

pikachu = Pokemon("Pikachu", 25, 35)
print(pikachu)

pikachu.take_damage(20)
print(pikachu)

pikachu.take_damage(20)
print(pikachu)

# Try to damage fainted Pokemon
pikachu.take_damage(10)

---

## **Benefits of Encapsulation**

In [ ]:
class Pokemon:
    def __init__(self, name, level):
        self.name = name
        self._level = level
        self._experience = 0
    
    def get_level(self):
        return self._level
    
    def gain_experience(self, amount):
        """Encapsulated experience system."""
        self._experience += amount
        
        # Internal logic hidden from user
        exp_needed = self._level * 100
        
        if self._experience >= exp_needed:
            self._level += 1
            self._experience = 0
            print(f"{self.name} leveled up to {self._level}!")
            return True
        return False

pikachu = Pokemon("Pikachu", 25)

# User doesn't need to know about experience formula
for i in range(30):
    pikachu.gain_experience(100)

print(f"\nFinal level: {pikachu.get_level()}")

print("\nBenefits:")
print("  1. Data validation")
print("  2. Internal logic hidden")
print("  3. Can change implementation without breaking code")
print("  4. Maintains invariants")

---

## **Practical: Fully Encapsulated Pokemon**

In [ ]:
class Pokemon:
    """Encapsulated Pokemon with data protection."""
    
    def __init__(self, name, level, hp, attack, defense):
        # Validate inputs
        if not name or not isinstance(name, str):
            raise ValueError("Name must be non-empty string")
        if not (1 <= level <= 100):
            raise ValueError("Level must be 1-100")
        if hp <= 0 or attack <= 0 or defense <= 0:
            raise ValueError("Stats must be positive")
        
        # Private attributes
        self._name = name
        self._level = level
        self._hp = hp
        self._max_hp = hp
        self._attack = attack
        self._defense = defense
        self._status = "Normal"
    
    # Read-only access
    def get_name(self):
        return self._name
    
    def get_level(self):
        return self._level
    
    def get_hp(self):
        return self._hp
    
    def get_stats(self):
        """Return copy of stats."""
        return {
            'name': self._name,
            'level': self._level,
            'hp': self._hp,
            'max_hp': self._max_hp,
            'attack': self._attack,
            'defense': self._defense,
            'status': self._status
        }
    
    # Controlled mutations
    def take_damage(self, damage):
        """Apply damage with validation."""
        if damage < 0:
            raise ValueError("Damage cannot be negative")
        
        if self._hp == 0:
            return "Already fainted"
        
        self._hp = max(0, self._hp - damage)
        
        if self._hp == 0:
            self._status = "Fainted"
            return f"{self._name} fainted!"
        
        return f"{self._name} took {damage} damage"
    
    def heal(self, amount):
        """Restore HP with validation."""
        if amount < 0:
            raise ValueError("Healing cannot be negative")
        
        if self._status == "Fainted":
            return "Cannot heal fainted Pokemon"
        
        old_hp = self._hp
        self._hp = min(self._max_hp, self._hp + amount)
        healed = self._hp - old_hp
        
        return f"{self._name} restored {healed} HP"
    
    def revive(self):
        """Revive fainted Pokemon."""
        if self._status != "Fainted":
            return f"{self._name} is not fainted"
        
        self._hp = self._max_hp // 2
        self._status = "Normal"
        return f"{self._name} was revived!"
    
    def __str__(self):
        return f"{self._name} Lv.{self._level} HP:{self._hp}/{self._max_hp} ({self._status})"

# Test encapsulation
print("Creating Pokemon:")
pikachu = Pokemon("Pikachu", 25, 35, 55, 40)
print(pikachu)

print("\nBattle:")
print(pikachu.take_damage(20))
print(pikachu)

print("\n" + pikachu.heal(10))
print(pikachu)

print("\nKnockout:")
print(pikachu.take_damage(50))
print(pikachu)

print("\nRevive:")
print(pikachu.revive())
print(pikachu)

# Try invalid operations
print("\nValidation:")
try:
    pikachu.take_damage(-10)
except ValueError as e:
    print(f"Caught: {e}")

try:
    invalid = Pokemon("", 150, 35, 55, 40)
except ValueError as e:
    print(f"Caught: {e}")

---

## **Practice Exercises**

### **Tasks 1-10: Standard exercises**

In [ ]:
# Practice exercises follow same pattern

---

## **Summary**

- Encapsulation bundles data and methods
- Controls access to internal state
- Prevents invalid states
- Use underscore `_attr` for private by convention
- Provide controlled interface methods
- Validate inputs in methods
- Hide implementation details
- Maintain class invariants
- Benefits: data integrity, flexibility, maintainability

---

## **Quick Reference**

```python
class Pokemon:
    def __init__(self, name, hp):
        # Private by convention
        self._name = name
        self._hp = hp
    
    # Controlled read access
    def get_hp(self):
        return self._hp
    
    # Controlled write access with validation
    def set_hp(self, value):
        if 0 <= value <= self._max_hp:
            self._hp = value
        else:
            raise ValueError("Invalid HP")
```